In [256]:
%run m3AndOfmMethods.ipynb

In [10]:
def addTargetsLossesColorPlot(ax, trained_models_per_checkpoint, model_name, cp_to_color, scalar_map):
    """
    Part of the Plot which shows all pretext and target losses in comparision, colored by the current epoch.
    """
    for epoch, data in trained_models_per_checkpoint.items(): 
        mean_losses = np.mean(data["xFold_losses"], axis=0)
        min_losses = np.min(data["xFold_losses"], axis=0)
        max_losses = np.max(data["xFold_losses"], axis=0)

        ax.plot(np.array(data["xFold_epochs"][0]), mean_losses, color=cp_to_color[epoch], linestyle="-", 
                 label=model_name)
        ax.fill_between(np.array(data["xFold_epochs"][0]), min_losses, max_losses, color=cp_to_color[epoch], alpha=0.5)

    ax.set_ylabel('Target Loss')
    ax.set_xlabel('Target Training Epochs')    
    scalar_map.set_array([]) 
    ax.grid(True)

In [52]:
def addTargetLossesPerPretextEpochColorPlot(ax, epochs_per_model, losses_per_model, rep_model_names):
    """
    Part of the Plot which shows all pretext and target losses in comparision, colored by the current epoch.
    """
    for i in range(len(rep_model_names)):
        epochs = epochs_per_model[i]
        losses = losses_per_model[i]
        model_name = rep_model_names[i]
             
        mean_losses = np.mean(losses, axis=0)
        min_losses = np.min(losses, axis=0)
        max_losses = np.max(losses, axis=0)
           
        cm = plt.get_cmap('rainbow') 
        cNorm  = matplot_colors.Normalize(vmin=0, vmax=np.max(epochs))
        scalar_map = cmx.ScalarMappable(norm=cNorm, cmap=cm)
            
        cp_to_color = {}
        for cp, mean_loss, max_loss, min_loss in zip(epochs, mean_losses, max_losses, min_losses): 
            c = scalar_map.to_rgba(cp)
            cp_to_color[cp] = c
            ax.plot(cp, mean_loss, color=c, linestyle="-", 
                     markeredgecolor='black', label=model_name, marker='o')
            
        color_min_max = 'gray'
        color_std = 'black'
        ax.errorbar(epochs, mean_losses, [mean_losses - min_losses, max_losses - mean_losses], 
                      fmt='.k', ecolor=color_min_max, lw=1, marker='', capsize=2)
        ax.errorbar(epochs, mean_losses, losses.std(0), fmt='.k', lw=1, marker='', ecolor=color_std, capsize=2)   
        
    ax.set_ylabel('Best Target Loss')
    ax.grid(True)
    return ax, cp_to_color, scalar_map   

In [53]:
def addPretextLossesColorPlot(ax, epochs, losses, model_name):
    """
    Part of the Plot which shows all pretext and target losses in comparision, colored by the current epoch.
    """
    mean_losses = np.mean(losses, axis=0)
    min_losses = np.min(losses, axis=0)
    max_losses = np.max(losses, axis=0)
   
    # Create a continuous norm to map from data points to colors.
    points = np.array([epochs, mean_losses]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    norm = plt.Normalize(0, epochs.max())
    
    # Plot everything.
    lc = LineCollection(segments, cmap='rainbow', norm=norm)
    lc.set_array(epochs)
    lc.set_linewidth(2)
    line = ax.add_collection(lc)
    
    ax.set_ylim(min_losses.min()*0.99, max_losses.max())
    ax.set_ylabel('Pretext Loss')
    ax.set_xlabel('Pretext Training Epochs')
    ax.grid(True)
    return ax 

In [5]:
def addM3BetweenCurvesPlot(ax, epochs1, epochs2, measurements1, measurements2, pretext_model_name, colors, linestyles, convergence_epoch2, use_interpolation=True):
    """
    The plot that shows the area between the pretext metric values and the target metric values. The area which corresponts to the MM3.
    """
    # Plot target and pretext curves.
    plot_mean_measurements1 = np.mean(measurements1, axis=0) 
    plot_mean_measurements2 = np.mean(measurements2, axis=0)  
    ax.plot(epochs1, plot_mean_measurements1, color=colors[pretext_model_name], linestyle=linestyles[pretext_model_name], markeredgecolor='black', marker='o')
    ax.plot(epochs2, plot_mean_measurements2, color=colors[pretext_model_name], linestyle=linestyles[pretext_model_name])
    
    # Interpolate the values for M3 calculation.   
    if use_interpolation:  
        measurements1, epochs = linearInterpolateMeasurements(measurements1, epochs1)
        measurements2, _ = linearInterpolateMeasurements(measurements2, epochs2)
        mean_measurements1 = np.mean(measurements1, axis=0) 
        mean_measurements2 = np.mean(measurements2, axis=0) 
    # Or only use the matching steps for M3.
    else:    
        matching_epoch_indices2 = np.where(np.in1d(epochs2, epochs1))
        matching_epoch_indices1 = np.where(np.in1d(epochs1, epochs2))
        epochs = epochs1[matching_epoch_indices1]
        measurements1 = [measurements1[j][matching_epoch_indices1] for j in range(len(measurements1))]
        measurements2 = [measurements2[j][matching_epoch_indices2] for j in range(len(measurements2))]
        mean_measurements1 = np.mean(measurements1, axis=0)
        mean_measurements2 = np.mean(measurements2, axis=0)
    
    # Calculate M3 values.  
    convergence_epoch_index2 = np.where(epochs == convergence_epoch2)[0][0] 
    mean_measurements1 = mean_measurements1[:convergence_epoch_index2+1]
    mean_measurements2 = mean_measurements2[:convergence_epoch_index2+1]   
    epochs = epochs[:convergence_epoch_index2+1]   
    mm3 = MM3(mean_measurements1, mean_measurements2)
    
    # Compare stability.
    mm3s = [MM3(measurements1[i][:convergence_epoch_index2+1], measurements2[i][:convergence_epoch_index2+1]) for i in range(len(measurements1))]
    mm3_plus = np.max(mm3s)-mm3
    mm3_minus = np.min(mm3s)-mm3
        
    # Plot everything.
    mean_measurements1 = mean_measurements1[:convergence_epoch_index2+1]
    mean_measurements2 = mean_measurements2[:convergence_epoch_index2+1]
    epochs = epochs[:convergence_epoch_index2+1]
    x = mean_measurements1 - mean_measurements2     
    positive_m3_indicies = np.where(x <= 0)
    negative_m3_indicies = np.where(x > 0)
    
    ax.fill_between(epochs[positive_m3_indicies], mean_measurements1[positive_m3_indicies], mean_measurements2[positive_m3_indicies], color='#6cb8bf', alpha=0.5)    
    ax.fill_between(epochs[negative_m3_indicies], mean_measurements1[negative_m3_indicies], mean_measurements2[negative_m3_indicies], color='#C70039' , alpha=0.5)

    ax.grid(True) 
    mm3_color ='#C70039' 
    if mm3 <= 0:
        mm3_color ='#6cb8bf'
    
    #line_offset = 5
    #ax.set_xlim(xmin=0, xmax=convergence_epoch2+line_offset)
    return mm3, mm3_plus, mm3_minus, mm3_color

In [2]:
def addM3Plot(ax, epochs1_per_xfold_per_model, epochs2_per_xfold_per_model, measurements1_per_xfold_per_model, measurements2_per_xfold_per_model, pretext_model_names, colors, linestyles, convergence_epoch2, use_interpolation=True):
    """
    The plot that shows the M3 during training for every metric-value pair of the pretext and target model.
    """
    mm3s=[]
    mm3s_minus=[]
    mm3s_plus=[]
    for i in range(len(pretext_model_names)): 

        # Interpolate the values for M3 calculation.   
        if use_interpolation:  
            measurements1_per_xfold, epochs = linearInterpolateMeasurements(measurements1_per_xfold_per_model[i], epochs1_per_xfold_per_model[i][0])
            measurements2_per_xfold, _ = linearInterpolateMeasurements(measurements2_per_xfold_per_model[i], epochs2_per_xfold_per_model[i][0])
        # Or only use matching steps for M3.
        else:    
            matching_epoch_indices2 = np.where(np.in1d(epochs2_per_xfold_per_model[i][0], epochs1_per_xfold_per_model[i][0]))
            matching_epoch_indices1 = np.where(np.in1d(epochs1_per_xfold_per_model[i][0], epochs2_per_xfold_per_model[i][0]))
            epochs = epochs1_per_xfold_per_model[i][0][matching_epoch_indices1]
            measurements1_per_xfold = [measurements1_per_xfold_per_model[i][j][matching_epoch_indices1] for j in range(len(measurements1_per_xfold_per_model[i]))]
            measurements2_per_xfold = [measurements2_per_xfold_per_model[i][j][matching_epoch_indices2] for j in range(len(measurements2_per_xfold_per_model[i]))]
           
        # Calculate the M3 values.  
        convergence_epoch_index2 = np.where(epochs == convergence_epoch2[i])[0][0] 
        measurements1_per_xfold = [measurements1_per_xfold[j][:convergence_epoch_index2+1] for j in range(len(measurements2_per_xfold))]
        measurements2_per_xfold = [measurements2_per_xfold[j][:convergence_epoch_index2+1] for j in range(len(measurements2_per_xfold))]
        epochs = epochs[:convergence_epoch_index2+1]
        
        mm3_per_epoch_per_xFold = np.subtract(measurements1_per_xfold, measurements2_per_xfold)    
        mm3_per_epoch = np.mean(mm3_per_epoch_per_xFold, axis=0)
        mm3_per_xFold = np.mean(mm3_per_epoch_per_xFold, axis=1)
        mm3 = np.mean(mm3_per_xFold)
        
        # Compare stability.
        min_mm3_per_epoch = np.min(mm3_per_epoch_per_xFold,axis=0)
        max_mm3_per_epoch = np.max(mm3_per_epoch_per_xFold,axis=0)
         
        mm3_plus = np.max(mm3_per_xFold)-mm3
        mm3_minus = np.min(mm3_per_xFold)-mm3
        
        # Plot everything.
        ax.plot(epochs, mm3_per_epoch, color=colors[pretext_model_names[i]], linestyle=linestyles[pretext_model_names[i]], label=pretext_model_names[i].split("R")[-1] + 
               ' MM3: ' + "{:.2f}% ".format(mm3) + "+" + "{:.2f}% ".format(mm3_plus) + "-" + "{:.2f}% ".format(abs(mm3_minus)))
        ax.fill_between(epochs, min_mm3_per_epoch, max_mm3_per_epoch, color=colors[pretext_model_names[i]], alpha=0.5)

        mm3s.append(mm3)
        mm3s_plus.append(mm3_plus)
        mm3s_minus.append(mm3_minus)
    ax.grid(True)
    return mm3s, mm3s_plus, mm3s_minus   

In [12]:
def addSM3orOFMBetweenCurvesPlot(ax, epochs, measurements, p_model_name, colors, linestyles, p_converget_epoch, use_interpolation=True, use_ofm=True):
    """
    The plot that shows the area between the minimal target metric value and the other target metric values. The area which corresponts to the MSM3 or MOFM.
    """
    # First we plot the measurements, if we use the OFM we plot them normalized. 
    plot_mean_measurements = np.mean(measurements, axis=0)
    if use_ofm:
        plot_mean_measurements = OFMNormalization(plot_mean_measurements, shift_measurements=True)     
    ax.plot(epochs, plot_mean_measurements, color=colors[p_model_name], linestyle=linestyles[p_model_name], 
            markeredgecolor='black', label=p_model_name, marker='o')
    
    # Only use matching steps for M3.
    mean_measurements = plot_mean_measurements
    # Or interpolate the values for SM3 or OFM calculation. 
    if use_interpolation:
        measurements, epochs = linearInterpolateMeasurements(measurements, epochs)
        mean_measurements = np.mean(measurements, axis=0)
    
    p_converget_epoch_index = np.where(epochs == p_converget_epoch)[0][0]    
    mean_measurements = mean_measurements[:p_converget_epoch_index+1]

    # Normalization for the OFM.
    if use_ofm:
        mean_measurements = OFMNormalization(mean_measurements, shift_measurements=True)     
    
    # Calculate SM3 or OFM values.
    msm3 = MSM3(mean_measurements)
    csm3 = cSM3(mean_measurements)
    max_sm3, max_sm3_index = mSM3(mean_measurements)     
    
    # Compare stability.
    if use_ofm:
        msm3s = [MSM3(OFMNormalization(measurements[i][:p_converget_epoch_index+1])) for i in range(len(measurements))]   
    else:
        msm3s = [MSM3(measurements[i][:p_converget_epoch_index+1]) for i in range(len(measurements))]   
    msm3_plus = np.max(msm3s)-msm3
    msm3_minus = np.min(msm3s)-msm3
    
    # Plot everything.
    min_measurement_index = np.argmin(mean_measurements)
    min_measurement = mean_measurements[min_measurement_index]
        
    ax.grid(True)  
    if msm3!=0:                    
        max_sm3_con = ConnectionPatch((epochs[max_sm3_index], min(mean_measurements[:max_sm3_index])), (epochs[max_sm3_index], mean_measurements[max_sm3_index]), "data", arrowstyle="<->", color='black', lw=2)
        csm3_con = ConnectionPatch((p_converget_epoch, min_measurement), (p_converget_epoch, min_measurement+csm3), "data", arrowstyle="<->", color='#C70039', lw=2)
        ax.add_artist(max_sm3_con)
        ax.add_artist(csm3_con)
        
        # This does not work for every case
        fill = np.arange(min_measurement_index)
        fill.fill(min_measurement)
        fill = np.concatenate((fill, mean_measurements[min_measurement_index:p_converget_epoch_index+1]))
        ax.fill_between(epochs[:p_converget_epoch_index+1], [min_measurement for i in range(p_converget_epoch_index+1)], 
                        fill, color='#C70039', alpha=0.5)
    #line_offset = 5
    #ax.set_xlim(xmin=-line_offset, xmax=p_converget_epoch+line_offset)
    #ax.set_ylim(ymin=min_measurement-5, ymax = np.max(mean_measurements)+5)  
    return msm3, msm3_plus, msm3_minus, csm3, max_sm3

In [1]:
def addSM3orOFMPlot(ax, epochs_per_xfold_per_model, measurements_per_xfold_per_model, pretext_model_names, colors, linestyles, p_converget_epoch, use_interpolation=True, use_ofm=True):
    """
    The plot that shows the SM3 or OFM during training for every metric-value of the target model.
    """
    msm3s=[]
    msm3s_minus=[]
    msm3s_plus=[]
    for i in range(len(pretext_model_names)): 
        
        # Only use matching steps for M3.
        epochs = epochs_per_xfold_per_model[i][0]
        measurements_per_xfold = measurements_per_xfold_per_model[i]
        # Or interpolate the values for SM3 or OFM calculation. 
        if use_interpolation:  
            measurements_per_xfold, epochs = linearInterpolateMeasurements(measurements_per_xfold, epochs)
            
        convergence_epoch_index = np.where(epochs == p_converget_epoch[i])[0][0] 
        measurements_per_xfold = [measurements_per_xfold[j][:convergence_epoch_index+1] for j in range(len(measurements_per_xfold))] 
        epochs = epochs[:convergence_epoch_index+1]

        # No Normalization for SM3.
        mean_measurements = np.mean(measurements_per_xfold, axis=0)  
        
        name = "MSM3"
        # Or normalization for the OFM.
        if use_ofm:
            name = "MOFM" 
            measurements_per_xfold = [OFMNormalization(measurements_per_xfold[j]) for j in range(len(measurements_per_xfold))]
            mean_measurements = OFMNormalization(mean_measurements)
        
        # Calculate SM3 or OFM values.
        msm3 = MSM3(mean_measurements)
        sm3_per_epoch = [SM3(k, mean_measurements) for k in range(len(mean_measurements))]
        sm3_per_epoch_per_xFold = []
        for j in range(len(measurements_per_xfold)):
            sm3s = [SM3(k, measurements_per_xfold[j]) for k in range(len(measurements_per_xfold[j]))]
            sm3_per_epoch_per_xFold.append(sm3s)    
            
        # Compare stability.
        msm3s_ = [MSM3(measurements_per_xfold[j]) for j in range(len(measurements_per_xfold))]    
        msm3_plus = np.max(msm3s_)-msm3
        msm3_minus = np.min(msm3s_)-msm3
                  
        min_sm3_per_epoch = np.min(sm3_per_epoch_per_xFold, axis=0)
        max_sm3_per_epoch = np.max(sm3_per_epoch_per_xFold, axis=0)
        
        # Plot everything.
        m_name = pretext_model_names[i].split("R")[-1] + ' ' + name +': ' + "{:.2f}%".format(msm3) + " +{:.2f}%".format(msm3_plus) + " -{:.2f}%".format(abs(msm3_minus))
        ax.plot(epochs, sm3_per_epoch, color=colors[pretext_model_names[i]], linestyle=linestyles[pretext_model_names[i]], label=m_name)
        ax.fill_between(epochs, min_sm3_per_epoch, max_sm3_per_epoch, color=colors[pretext_model_names[i]], alpha=0.5)
        
        msm3s.append(msm3)
        msm3s_plus.append(msm3_plus)
        msm3s_minus.append(msm3_minus)

    ax.grid(True)
    return msm3s, msm3s_plus, msm3s_minus 

In [ ]:
def addMetricPlot(ax, t_epochs_per_xFold_per_model, t_best_losses_per_xFold_per_model, model_names, colors, linestyles):
    """
    Simple plot to show the curve of a metric during training.
    """
    for i in range(len(model_names)):
        epochs = t_epochs_per_xFold_per_model[i][0]
        losses = t_best_losses_per_xFold_per_model[i]
        model_name = model_names[i]
        
        mean_losses = np.mean(losses, axis=0)
        min_losses = np.min(losses, axis=0)
        max_losses = np.max(losses, axis=0)
        
        ax.plot(epochs, mean_losses, color=colors[model_name], linestyle=linestyles[model_name], 
                 markeredgecolor='black', label=model_name, marker='o')
        ax.fill_between(epochs, min_losses, max_losses, color=colors[model_name], alpha=0.5)
        
    ax.grid(True)
    return ax   

In [ ]:
def addPretextMetricPlot(ax, p_epochs_per_xFold_per_model, p_best_metric_per_xFold_per_model, pretext_model_names, colors, linestyles):
    """
    Simple plot to show the curve of a metric of a pretext model during training.
    """
    for i in range(len(pretext_model_names)):
        epochs = p_epochs_per_xFold_per_model[i][0]
        values = p_best_metric_per_xFold_per_model[i]
        model_name = pretext_model_names[i]
        
        mean_values = np.mean(values, axis=0)
        min_values = np.min(values, axis=0)
        max_values = np.max(values, axis=0)

        ax.plot(epochs, mean_values, color=colors[model_name], linestyle=linestyles[model_name],label=model_name)
        ax.fill_between(epochs, min_values, max_values, color=colors[model_name], alpha=0.5)
        
    ax.grid(True)
    return ax   